In [1]:
from transformers import AutoTokenizer
import tritonhttpclient
import numpy as np
import cv2

/Users/camilosanta/vearing/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/camilosanta/vearing/lib/python3.8/site-packages/tritonhttpclient/__init__.py:31: DeprecationWarning: The package `tritonhttpclient` is deprecated and will be removed in a future version. Please use instead `tritonclient.http`
  warnings.warn(


In [2]:
url='44.206.242.119:8000'
triton_client = tritonhttpclient.InferenceServerClient(url=url, verbose=False)

In [3]:
img=cv2.imread('/Users/camilosanta/Downloads/baby.jpeg')
face=img[50:170,120:240]
face=cv2.resize(face,(48,48))
gray = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY).astype(np.float32)
gray=gray[np.newaxis,:,:,np.newaxis]
img=cv2.resize(img,(513,513))
img=img[np.newaxis,:,:,:]
img.shape,face.shape,gray.shape

((1, 513, 513, 3), (48, 48, 3), (1, 48, 48, 1))

In [4]:
input_name='conv2d_8_input'
output_name = 'dense_5'
model_name='face-emotion'
model_version='1'
input0 = tritonhttpclient.InferInput(input_name, (1, 48,48,1), 'FP32')
input0.set_data_from_numpy(gray, binary_data=False)
output = tritonhttpclient.InferRequestedOutput(output_name, binary_data=False)
response = triton_client.infer(model_name,model_version=model_version,inputs=[input0],outputs=[output])
logits = response.as_numpy(output_name)
logits

array([[0., 0., 0., 0., 0., 0., 1.]], dtype=float32)

In [5]:
input_name='ImageTensor'
output_name = 'ResizeBilinear_3'
model_name='face-bokeh'
model_version='1'
input0 = tritonhttpclient.InferInput(input_name, (1,513,513,3), 'UINT8')
input0.set_data_from_numpy(img, binary_data=False)
output = tritonhttpclient.InferRequestedOutput(output_name, binary_data=False)
response = triton_client.infer(model_name,model_version=model_version,inputs=[input0],outputs=[output])
segmap = response.as_numpy(output_name)
segmap.shape

(1, 513, 513, 21)